"""

This notebook runs through Section 2.3: Style Transfer. 
The images generated are similar to those in Figure 3. 

The Directory Paths block should be edited to suit local directory structure. 
The Chosen Parameters block can be changed to try out different experimental settings. 

"""

In [2]:
import cv2
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import shutil
import tensorflow as tf
import tensorflow.contrib.slim as slim

from utils.imaging import display_image, format_image, save_image
import vgg

3.5.2 |Anaconda 4.2.0 (64-bit)| (default, Jul  5 2016, 11:41:13) [MSC v.1900 64 bit (AMD64)]


### Directory Paths

In [ ]:
# Path to image we are extracting content from
real_image_path = './coastal_scene.jpg'

# Path to vgg19 checkpoint, must be downloaded separately
checkpoint_path = './vgg_19.ckpt'

# Location of tensorboard summaries
tensorboard_dir = './train/'

# Path to directory used for storing images
debug_dir = './debug/'

# Determines whether information is saved between runs
# for tensorboard
reset_saves = True
if reset_saves is True:
    # Ensure tensorboard is not running when you try to delete
    # this directory
    if os.path.exists(tensorboard_dir):
        shutil.rmtree(tensorboard_dir)
        
# Create the debug directory if it doesn't exist
# Tensorboard directory is made automatically if it doesn't exist
if os.path.exists(debug_dir):
    shutil.rmtree(debug_dir)
os.makedirs(debug_dir)

### Chosen Parameters

In [ ]:
# Dimensions desired for image, channels must be kept as 3
height = 224
width = 224
channels = 3

# Set the seeds to provide consistency between runs
# Can also comment out for variability between runs
np.random.seed(2)
tf.set_random_seed(2)

# Layer being used to produce features
content_layer = 'vgg_19/conv2/conv2_2'
style_list = ['vgg_19/conv1/conv1_1', 'vgg_19/conv2/conv2_1', 'vgg_19/conv3/conv3_1',
    'vgg_19/conv4/conv4_1', 'vgg_19/conv5/conv5_1']

# Chosen depth corresponds to how many feature layers you want to use
# for the style component
chosen_depth = 2

# Learning rate for optimizer
learning_rate = 1e-1

# Number of training and validation step
# In this instance, validation refers to when we would like to examine the 
# currently optimized image, save it, and loss
training_steps = 100000
validation_steps = 1000

# Online debugging refers to images that will be displayed within the notebook 
# using plt
# Offline debugging refers to images that will be saved to folder using plt
debug_online = True
debug_offline = True

### Set up the input node and feature extractor

In [ ]:
# The input node to the graph
# These values are what is required by vgg19 for height, width, channels
input_var_initial_value = np.random.rand(1, height, width, channels)
input_var = tf.Variable(input_var_initial_value, dtype=tf.float32, name='input_var')

# Load the vgg model
with slim.arg_scope(vgg.vgg_arg_scope()):
    end_points = vgg.vgg_19_conv(input_var)

### Load real image and create to-be-optimized image

In [ ]:
# Construct the real image tensor
# And the graph operation which assigns it to input_var
real_image = plt.imread(real_image_path)
real_image = cv2.resize(real_image, (height, width))
real_image_batch = np.expand_dims(real_image, axis=0)
real_image_batch = np.asarray(real_image_batch, dtype=np.float32)
real_image_tensor = tf.Variable(real_image_batch, dtype=tf.float32, name='real_image')

assign_real_image = tf.assign(input_var, real_image_tensor, name='assign_real_image')

# Construct the white noise tensor
# And the graph operation which assigns it to input_var
white_noise = np.random.rand(height, width, channels) * 255.
white_noise_batch = np.expand_dims(white_noise, axis=0)
white_noise_batch = np.asarray(white_noise_batch, dtype=np.float32)
white_noise_tensor = tf.Variable(white_noise_batch, dtype=tf.float32, name='white_noise')

assign_white_noise = tf.assign(input_var, white_noise_tensor, name='assign_white_noise')